In [1]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate();
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [2]:
import pandas as pd
import tweepy

In [3]:
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Following Count:{ith_tweet[1]}")
    print(f"Follower Count:{ith_tweet[2]}")
    print(f"Total Tweets:{ith_tweet[3]}")
    print(f"Retweet Count:{ith_tweet[4]}")

In [4]:
def scrape(words, date_since, numtweet):
    db = pd.DataFrame(columns=['username','following','followers', 'totaltweets', 'retweetcount'])
    tweets = tweepy.Cursor(api.search, q=words, lang="en",since=date_since, tweet_mode='extended').items(numtweet)
    list_tweets = [tweet for tweet in tweets]
    i = 1  
    for tweet in list_tweets:
        username = tweet.user.screen_name
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        ith_tweet = [username,following,followers,totaltweets, retweetcount]
        db.loc[len(db)] = ith_tweet
        printtweetdata(i, ith_tweet)
        i = i+1
    filename = 'ScrapedTweets.csv'
    db.to_csv(filename)

In [5]:
consumer_key = 'zsTrAIqVMAl6fMjd3aCnc243e'
consumer_secret = 'bezIY6xoFrQTVr0BYdinyyd8eVc60JNDaXMCWFXr3blfVZ9PV0'
access_key = '1059163618317606912-u4xy1i6vU5eptnQxnka9HXvz2eCNX6'
access_secret = 'YQFGmkCYKIyDLmMO4Nfw3iRjPAMY9E4MtfLacmgohOsSI'

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
words = "ELon Musk"
date_since = "2020-08-10"
numtweet = 100

In [8]:
scrape(words, date_since, numtweet)


Tweet 1:
Username:Smack_Check
Following Count:891
Follower Count:848
Total Tweets:14532
Retweet Count:0

Tweet 2:
Username:karl_popp
Following Count:952
Follower Count:1838
Total Tweets:196446
Retweet Count:0

Tweet 3:
Username:mr_awotwe
Following Count:179
Follower Count:119
Total Tweets:1026
Retweet Count:1449

Tweet 4:
Username:C_Gritz
Following Count:752
Follower Count:3689
Total Tweets:33423
Retweet Count:0

Tweet 5:
Username:andredarmanin
Following Count:1153
Follower Count:2304
Total Tweets:84327
Retweet Count:0

Tweet 6:
Username:Wordsnflowers1
Following Count:769
Follower Count:542
Total Tweets:92182
Retweet Count:21

Tweet 7:
Username:RicoBro31
Following Count:46
Follower Count:6
Total Tweets:9
Retweet Count:2774

Tweet 8:
Username:iamsameer1009
Following Count:193
Follower Count:202
Total Tweets:28569
Retweet Count:1449

Tweet 9:
Username:freedoms2030
Following Count:549
Follower Count:263
Total Tweets:7864
Retweet Count:2774

Tweet 10:
Username:Fraga125
Following Count:125

In [9]:
csv = spark.read.csv('ScrapedTweets.csv', inferSchema=True, header=True)
csv.show(10)

+---+--------------+---------+---------+-----------+------------+
|_c0|      username|following|followers|totaltweets|retweetcount|
+---+--------------+---------+---------+-----------+------------+
|  0|   Smack_Check|      891|      848|      14532|           0|
|  1|     karl_popp|      952|     1838|     196446|           0|
|  2|     mr_awotwe|      179|      119|       1026|        1449|
|  3|       C_Gritz|      752|     3689|      33423|           0|
|  4| andredarmanin|     1153|     2304|      84327|           0|
|  5|Wordsnflowers1|      769|      542|      92182|          21|
|  6|     RicoBro31|       46|        6|          9|        2774|
|  7| iamsameer1009|      193|      202|      28569|        1449|
|  8|  freedoms2030|      549|      263|       7864|        2774|
|  9|      Fraga125|      125|       88|       2358|        1449|
+---+--------------+---------+---------+-----------+------------+
only showing top 10 rows

